In [36]:
# Cell 1: Install Libraries
# (Uncomment the next line if you need to reinstall packages in this kernel)
# !pip install vectorbt yfinance pandas numpy

import pandas as pd
import numpy as np
import yfinance as yf
import vectorbt as vbt
from datetime import datetime, timedelta
import time
import sys

print("Libraries imported successfully.")
print(f"Python executable: {sys.executable}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"VectorBT version: {vbt.__version__}")
print(f"YFinance version: {yf.__version__}")

# Safe, local indicator and backtest implementations to avoid pandas_ta
def _ensure_series(s, name=None, index=None):
    # If DataFrame, take first column
    if isinstance(s, pd.DataFrame):
        if s.shape[1] == 1:
            s = s.iloc[:, 0]
        else:
            # prefer column named 'Close' or first column
            if 'Close' in s.columns:
                s = s['Close']
            else:
                s = s.iloc[:, 0]
    # If index mismatch, reindex to provided index
    if index is not None:
        s = s.reindex(index)
    s = s.astype(float)
    return s


def calculate_indicators_safe(ohlc, ema_short, ema_long, adx_len):
    # EMAs using vectorbt; their .ma can return Series or DataFrame
    ema_s = vbt.MA.run(ohlc['Close'], ema_short, ewm=True).ma
    ema_l = vbt.MA.run(ohlc['Close'], ema_long, ewm=True).ma

    # Ensure they are Series aligned to ohlc.index
    ema_s = _ensure_series(ema_s, name='ema_s', index=ohlc.index)
    ema_l = _ensure_series(ema_l, name='ema_l', index=ohlc.index)

    # Custom ADX implementation (avoids pandas_ta compatibility issues)
    high = ohlc['High']
    low = ohlc['Low']
    close = ohlc['Close']

    prev_high = high.shift(1)
    prev_low = low.shift(1)
    prev_close = close.shift(1)

    tr1 = high - low
    tr2 = (high - prev_close).abs()
    tr3 = (low - prev_close).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    up_move = high - prev_high
    down_move = prev_low - low

    plus_dm = up_move.where((up_move > down_move) & (up_move > 0), 0.0)
    minus_dm = down_move.where((down_move > up_move) & (down_move > 0), 0.0)

    # Wilder smoothing via exponential with alpha=1/period (approximation)
    atr = tr.ewm(alpha=1/adx_len, adjust=False).mean()
    sm_plus = plus_dm.ewm(alpha=1/adx_len, adjust=False).mean()
    sm_minus = minus_dm.ewm(alpha=1/adx_len, adjust=False).mean()

    plus_di = 100 * (sm_plus / atr)
    minus_di = 100 * (sm_minus / atr)

    # Protect division by zero
    denom = (plus_di + minus_di).replace(0, np.nan)
    dx = (plus_di - minus_di).abs() / denom * 100
    dx = dx.fillna(0)
    adx = dx.ewm(alpha=1/adx_len, adjust=False).mean()

    adx = _ensure_series(adx, name='adx', index=ohlc.index)

    return ema_s, ema_l, adx


def run_backtest_safe(data, trade_size, ema_short_local=None, ema_long_local=None, adx_len_local=None):
    # Allow optional overrides but default to notebook globals
    if ema_short_local is None:
        ema_short_local = globals().get('ema_short')
    if ema_long_local is None:
        ema_long_local = globals().get('ema_long')
    if adx_len_local is None:
        adx_len_local = globals().get('adx_len')

    if data is None or data.empty:
        raise ValueError('Input data is empty. Fetch data before running backtest.')

    ema_s, ema_l, adx = calculate_indicators_safe(data, ema_short_local, ema_long_local, adx_len_local)

    # Debug prints for shapes and index
    print('ema_s type/shape:', type(ema_s), getattr(ema_s, 'shape', None))
    print('ema_l type/shape:', type(ema_l), getattr(ema_l, 'shape', None))
    print('adx type/shape:', type(adx), getattr(adx, 'shape', None))
    print('data Close shape:', data['Close'].shape)

    # Ensure boolean masks are Series aligned to data['Close']
    adx_threshold_local = globals().get('adx_threshold', 25)
    buy = (ema_s > ema_l) & (adx > adx_threshold_local)
    sell = (ema_s < ema_l) & (adx > adx_threshold_local)

    # Debug: counts
    print('buy True count:', int(buy.sum()))
    print('sell True count:', int(sell.sum()))

    buy = buy.reindex(data.index).fillna(False)
    sell = sell.reindex(data.index).fillna(False)

    # Apply leverage to initial cash so strategy can open leveraged positions
    effective_init_cash = globals().get('initial_balance', 100.0) * globals().get('leverage', 1.0)
    print('Using effective_init_cash:', effective_init_cash)

    portfolio = vbt.Portfolio.from_signals(
        data['Close'], entries=buy, exits=sell,
        size=trade_size, init_cash=effective_init_cash,
        sl_stop=globals().get('sl_percent', 0.2)/100, tp_stop=globals().get('tp_percent', 0.4)/100,
        cash_sharing=True, freq=globals().get('timeframe', '1h')
    )
    return portfolio

# Expose safe names
calculate_indicators = calculate_indicators_safe
run_backtest = run_backtest_safe
print('Safe indicator/backtest functions are defined in this kernel.')

Libraries imported successfully.
Python executable: c:\Users\Dax\anaconda3\envs\trading\python.exe
NumPy version: 2.2.6
Pandas version: 2.3.3
VectorBT version: 0.28.1
YFinance version: 0.2.66
Safe indicator/backtest functions are defined in this kernel.


In [2]:
# --- Option 3: ADX + EMA Trend Strategy ---
end_date = datetime.now()
start_date = end_date - timedelta(days=729)
timeframe = '1h'

initial_balance = 100.0
leverage = 200.0
eurusd_lot_size = 0.05
xauusd_lot_size = 0.01

forex_contract_size = 100000
gold_contract_size = 100

eurusd_trade_size = eurusd_lot_size * forex_contract_size
xauusd_trade_size = xauusd_lot_size * gold_contract_size

ema_short = 20
ema_long = 50
adx_len = 14
adx_threshold = 25
tp_percent = 0.4
sl_percent = 0.2

print("Option 3: ADX + EMA Config Loaded")


Option 3: ADX + EMA Config Loaded


In [3]:
# Cell 3: The Advanced Data Fetching Function

MAX_DAYS_INTRADAY = 730 # Max days for a single yfinance intraday request
SLEEP_BETWEEN_CALLS = 1   # Pause for 1 second between API calls

def _days_between(a, b):
    return (b - a).days

def split_date_ranges(start_dt, end_dt, max_days=MAX_DAYS_INTRADAY):
    ranges = []
    cur_start = start_dt
    while cur_start < end_dt:
        cur_end = min(cur_start + timedelta(days=max_days), end_dt)
        ranges.append((cur_start, cur_end))
        cur_start = cur_end
    return ranges

def fetch_data_chunked(ticker, start, end, interval="1h", auto_adjust=True, debug=True):
    start_dt = pd.to_datetime(start)
    end_dt = pd.to_datetime(end)

    intraday = interval not in ("1d", "5d", "1wk", "1mo")
    total_days = _days_between(start_dt, end_dt)
    if debug:
        print(f"\nFetching {interval} data for {ticker} from {start_dt.date()} to {end_dt.date()} ({total_days} days)")

    date_ranges = [(start_dt, end_dt)]
    if intraday and total_days > MAX_DAYS_INTRADAY:
        date_ranges = split_date_ranges(start_dt, end_dt, max_days=MAX_DAYS_INTRADAY)
        if debug:
            print(f"Splitting into {len(date_ranges)} chunks (max {MAX_DAYS_INTRADAY} days each).")

    pieces = []
    for i, (s, e) in enumerate(date_ranges, 1):
        e_for_yf = e + timedelta(days=1) # yfinance end date is exclusive
        if debug:
            print(f"  Chunk {i}: {s.date()} -> {e.date()} (requesting end={e_for_yf.date()})")
        try:
            df = yf.download(
                ticker,
                start=s.strftime("%Y-%m-%d"),
                end=e_for_yf.strftime("%Y-%m-%d"),
                interval=interval,
                progress=False,
                auto_adjust=auto_adjust,
                threads=True
            )
            if df is None or df.empty:
                print(f"    -> empty result for this chunk.")
                continue
            pieces.append(df)
            print(f"    -> fetched {len(df)} rows.")
        except Exception as ex:
            print(f"    -> error fetching chunk {i}: {ex}")
        time.sleep(SLEEP_BETWEEN_CALLS)

    if not pieces:
        print("Failed to fetch any data.")
        return None

    result = pd.concat(pieces)
    result = result[~result.index.duplicated(keep="first")]
    result = result.sort_index()
    print(f"\nSuccessfully fetched total of {len(result)} rows for {ticker}.")
    return result

print("Chunked data fetching function is defined.")

Chunked data fetching function is defined.


In [37]:
# Cell 4: Fetch Data for All Assets (No Changes Needed)

# The chunking function will now work correctly with the updated date range.
eurusd_data = fetch_data_chunked('EURUSD=X', start_date, end_date, timeframe)
# FIX: Using the correct ticker for Gold Futures
xauusd_data = fetch_data_chunked('GC=F', start_date, end_date, timeframe)

if eurusd_data is not None:
    print("\nEUR/USD Data Head:")
    display(eurusd_data.head())


Fetching 1h data for EURUSD=X from 2023-10-20 to 2025-10-18 (729 days)
  Chunk 1: 2023-10-20 -> 2025-10-18 (requesting end=2025-10-19)
    -> fetched 12331 rows.
    -> fetched 12331 rows.

Successfully fetched total of 12331 rows for EURUSD=X.

Fetching 1h data for GC=F from 2023-10-20 to 2025-10-18 (729 days)
  Chunk 1: 2023-10-20 -> 2025-10-18 (requesting end=2025-10-19)

Successfully fetched total of 12331 rows for EURUSD=X.

Fetching 1h data for GC=F from 2023-10-20 to 2025-10-18 (729 days)
  Chunk 1: 2023-10-20 -> 2025-10-18 (requesting end=2025-10-19)
    -> fetched 11476 rows.
    -> fetched 11476 rows.

Successfully fetched total of 11476 rows for GC=F.

EUR/USD Data Head:

Successfully fetched total of 11476 rows for GC=F.

EUR/USD Data Head:


Price,Close,High,Low,Open,Volume
Ticker,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X
Datetime,,,,,
2023-10-19 23:00:00+00:00,1.058985,1.058985,1.058201,1.058313,0
2023-10-20 00:00:00+00:00,1.057865,1.058873,1.057753,1.058873,0
2023-10-20 01:00:00+00:00,1.057530,1.057865,1.057194,1.057865,0
2023-10-20 02:00:00+00:00,1.058089,1.058201,1.057418,1.057418,0
2023-10-20 03:00:00+00:00,1.057865,1.058201,1.057753,1.058089,0


In [38]:
# Cell 5: The Backtesting Engine (Final, Corrected Version)

def calculate_indicators(ohlc, ema_short, ema_long, adx_len):
    ema_s = vbt.MA.run(ohlc['Close'], ema_short, ewm=True).ma
    ema_l = vbt.MA.run(ohlc['Close'], ema_long, ewm=True).ma
    adx = vbt.IndicatorFactory.from_pandas_ta('adx')(ohlc['High'], ohlc['Low'], ohlc['Close'], length=adx_len).ADX_14
    return ema_s, ema_l, adx

def run_backtest(data, trade_size):
    close = data['Close']
    ema_s, ema_l, adx = calculate_indicators(data, ema_short, ema_long, adx_len)

    buy = (ema_s > ema_l) & (adx > adx_threshold)
    sell = (ema_s < ema_l) & (adx > adx_threshold)

    portfolio = vbt.Portfolio.from_signals(
        close, entries=buy, exits=sell,
        size=trade_size, init_cash=initial_balance,
        sl_stop=sl_percent/100, tp_stop=tp_percent/100,
        cash_sharing=True, freq=timeframe
    )
    return portfolio


In [27]:
# !pip install pandas-ta

In [39]:
# Cell 6: Run & Analyze EUR/USD Backtest (Using safe implementation)

print('Running EURUSD backtest using run_backtest_safe from cell 1')
try:
    eurusd_portfolio = run_backtest_safe(eurusd_data, eurusd_trade_size)
    print('EURUSD backtest (safe) completed successfully')
except Exception as e:
    print(f"Error running EUR/USD backtest (safe): {type(e).__name__}: {e}")
    raise

if eurusd_portfolio is not None:
    print('\n--- EUR/USD Performance ---')
    print(eurusd_portfolio.stats())

Running EURUSD backtest using run_backtest_safe from cell 1
ema_s type/shape: <class 'pandas.core.series.Series'> (12331,)
ema_l type/shape: <class 'pandas.core.series.Series'> (12331,)
adx type/shape: <class 'pandas.core.series.Series'> (12331,)
data Close shape: (12331, 1)
buy True count: 0
sell True count: 0
Using effective_init_cash: 20000.0
EURUSD backtest (safe) completed successfully

--- EUR/USD Performance ---
ema_s type/shape: <class 'pandas.core.series.Series'> (12331,)
ema_l type/shape: <class 'pandas.core.series.Series'> (12331,)
adx type/shape: <class 'pandas.core.series.Series'> (12331,)
data Close shape: (12331, 1)
buy True count: 0
sell True count: 0
Using effective_init_cash: 20000.0
EURUSD backtest (safe) completed successfully

--- EUR/USD Performance ---
Start                         2023-10-19 23:00:00+00:00
End                           2025-10-17 21:00:00+00:00
Period                                513 days 19:00:00
Start Value                                   

In [40]:
# Cell 7: Plot EUR/USD Portfolio

import plotly.graph_objects as go
import vectorbt as vbt

# Option 1: Default Vectorbt Plot (Line Plot with Trades)
fig_eur = eurusd_portfolio.plot(
    subplots=['trades', 'cum_returns', 'drawdowns'],  # Subplots: trades on price, cumulative returns, drawdowns
    subplot_settings={
        'trades': {
            'close_trace_kwargs': dict(  # Style the price line
                name='EUR/USD Price',
                line=dict(color='blue', width=1),
                mode='lines'
            ),
            'add_trace_kwargs': dict(row=1, col=1)  # Position trades on price subplot
        },
        'cum_returns': {
            'title': 'Cumulative Returns',
            'yaxis_kwargs': dict(title='Returns (%)')
        },
        'drawdowns': {
            'title': 'Drawdowns',
            'yaxis_kwargs': dict(title='Drawdown (%)')
        }
    },
    make_subplots_kwargs=dict(
        rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05
    ),
    show_titles=True
)
print("EUR/USD Portfolio Plot (Line):")
fig_eur.show()

# Option 2: OHLC Candlestick Plot with Trades
def plot_ohlc_eur(pf, column=None, add_trace_kwargs=None, fig=None):
    ohlc_df = pd.DataFrame({
        'Open': pf.open.get(column=column),
        'High': pf.high.get(column=column),
        'Low': pf.low.get(column=column),
        'Close': pf.close.get(column=column)
    })
    candlestick = go.Candlestick(
        x=ohlc_df.index,
        open=ohlc_df['Open'],
        high=ohlc_df['High'],
        low=ohlc_df['Low'],
        close=ohlc_df['Close'],
        name='EUR/USD OHLC',
        increasing_line_color='green',
        decreasing_line_color='red'
    )
    fig.add_trace(candlestick, **(add_trace_kwargs or {}))

fig_eur_ohlc = eurusd_portfolio.plot(
    subplots=[
        ('ohlc', dict(
            title='EUR/USD OHLC Price',
            yaxis_kwargs=dict(title='Price'),
            plot_func=plot_ohlc_eur,
            pass_add_trace_kwargs=True,
            check_is_not_grouped=True
        )),
        'trades',
        'cum_returns',
        'drawdowns'
    ],
    subplot_settings={
        'ohlc': {'add_trace_kwargs': dict(row=1, col=1)},
        'trades': {'add_trace_kwargs': dict(row=1, col=1)},  # Overlay trades on OHLC
        'cum_returns': {'yaxis_kwargs': dict(title='Returns (%)')},
        'drawdowns': {'yaxis_kwargs': dict(title='Drawdown (%)')}
    },
    make_subplots_kwargs=dict(
        rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05
    ),
    show_titles=True
)
print("EUR/USD Portfolio Plot (OHLC):")
fig_eur_ohlc.show()

c:\Users\Dax\anaconda3\envs\trading\Lib\site-packages\vectorbt\generic\plots_builder.py:339: UserWarning:

Subplot 'trades' does not support grouped data



ImportError: Please install anywidget to use the FigureWidget class

In [42]:
# Cell 8: Run & Analyze XAU/USD (Gold) Backtest (Corrected)

print('Running XAUUSD backtest using run_backtest_safe')
try:
    xauusd_portfolio = run_backtest_safe(xauusd_data, xauusd_trade_size)
    print('XAUUSD backtest (safe) completed successfully')
except Exception as e:
    print(f"Error running XAU/USD backtest (safe): {type(e).__name__}: {e}")
    # Provide None so plotting cell can handle absence
    xauusd_portfolio = None

if xauusd_portfolio is not None:
    print('\n--- XAU/USD Performance ---')
    print(xauusd_portfolio.stats())
else:
    print('XAUUSD portfolio not available; skipping stats.')

Running XAUUSD backtest using run_backtest_safe
ema_s type/shape: <class 'pandas.core.series.Series'> (11476,)
ema_l type/shape: <class 'pandas.core.series.Series'> (11476,)
adx type/shape: <class 'pandas.core.series.Series'> (11476,)
data Close shape: (11476, 1)
buy True count: 0
sell True count: 0
Using effective_init_cash: 20000.0
XAUUSD backtest (safe) completed successfully

--- XAU/USD Performance ---
ema_s type/shape: <class 'pandas.core.series.Series'> (11476,)
ema_l type/shape: <class 'pandas.core.series.Series'> (11476,)
adx type/shape: <class 'pandas.core.series.Series'> (11476,)
data Close shape: (11476, 1)
buy True count: 0
sell True count: 0
Using effective_init_cash: 20000.0
XAUUSD backtest (safe) completed successfully

--- XAU/USD Performance ---
Start                         2023-10-20 04:00:00+00:00
End                           2025-10-17 20:00:00+00:00
Period                                478 days 04:00:00
Start Value                                     20000.0
En

In [44]:
# Cell 9: Plot XAU/USD Portfolio

import plotly.graph_objects as go
import vectorbt as vbt

# Option 1: Default Vectorbt Plot (Line Plot with Trades)
fig_xau = xauusd_portfolio.plot(
    subplots=['trades', 'cum_returns', 'drawdowns'],
    subplot_settings={
        'trades': {
            'close_trace_kwargs': dict(
                name='XAU/USD Price',
                line=dict(color='gold', width=1),
                mode='lines'
            ),
            'add_trace_kwargs': dict(row=1, col=1)
        },
        'cum_returns': {
            'title': 'Cumulative Returns',
            'yaxis_kwargs': dict(title='Returns (%)')
        },
        'drawdowns': {
            'title': 'Drawdowns',
            'yaxis_kwargs': dict(title='Drawdown (%)')
        }
    },
    make_subplots_kwargs=dict(
        rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05
    ),
    show_titles=True
)
print("XAU/USD Portfolio Plot (Line):")
fig_xau.show()

# Option 2: OHLC Candlestick Plot with Trades
def plot_ohlc_xau(pf, column=None, add_trace_kwargs=None, fig=None):
    ohlc_df = pd.DataFrame({
        'Open': pf.open.get(column=column),
        'High': pf.high.get(column=column),
        'Low': pf.low.get(column=column),
        'Close': pf.close.get(column=column)
    })
    candlestick = go.Candlestick(
        x=ohlc_df.index,
        open=ohlc_df['Open'],
        high=ohlc_df['High'],
        low=ohlc_df['Low'],
        close=ohlc_df['Close'],
        name='XAU/USD OHLC',
        increasing_line_color='green',
        decreasing_line_color='red'
    )
    fig.add_trace(candlestick, **(add_trace_kwargs or {}))

fig_xau_ohlc = xauusd_portfolio.plot(
    subplots=[
        ('ohlc', dict(
            title='XAU/USD OHLC Price',
            yaxis_kwargs=dict(title='Price'),
            plot_func=plot_ohlc_xau,
            pass_add_trace_kwargs=True,
            check_is_not_grouped=True
        )),
        'trades',
        'cum_returns',
        'drawdowns'
    ],
    subplot_settings={
        'ohlc': {'add_trace_kwargs': dict(row=1, col=1)},
        'trades': {'add_trace_kwargs': dict(row=1, col=1)},
        'cum_returns': {'yaxis_kwargs': dict(title='Returns (%)')},
        'drawdowns': {'yaxis_kwargs': dict(title='Drawdown (%)')}
    },
    make_subplots_kwargs=dict(
        rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05
    ),
    show_titles=True
)
print("XAU/USD Portfolio Plot (OHLC):")
fig_xau_ohlc.show()

ImportError: Please install anywidget to use the FigureWidget class

In [ ]:
# Diagnostic: inspect indicators and signal masks for EURUSD
ema_s, ema_l, adx = calculate_indicators_safe(eurusd_data, ema_short, ema_long, adx_len)

# Basic stats
print('EMA short/long last values:', ema_s.iloc[-1], ema_l.iloc[-1])
print('ADX last value:', adx.iloc[-1])
print('\nADX stats: min={:.2f}, mean={:.2f}, max={:.2f}'.format(adx.min(), adx.mean(), adx.max()))

# Raw crossover counts (without ADX)
ema_longer = ema_l
ema_shorter = ema_s
cross_up = (ema_shorter > ema_longer) & (~(ema_shorter > ema_longer).shift(1).fillna(False))
cross_down = (ema_shorter < ema_longer) & (~(ema_shorter < ema_longer).shift(1).fillna(False))
print('Raw EMA cross up count:', int(cross_up.sum()))
print('Raw EMA cross down count:', int(cross_down.sum()))

# How many times ADX is above threshold
adx_over = (adx > adx_threshold)
print('ADX > threshold count:', int(adx_over.sum()))

# Combined condition count (what we currently require)
combined = (ema_shorter > ema_longer) & (adx > adx_threshold)
print('Combined buy condition count:', int(combined.sum()))

# Show first 10 timestamps where combined would be True (if any)
if combined.sum() > 0:
    print('\nSample combined buy timestamps:')
    print(eurusd_data.index[combined].tolist()[:10])
else:
    print('\nNo combined buy signals found with current parameters.')

# Show a small table of recent indicator values for manual inspection
display(pd.DataFrame({'Close': eurusd_data['Close'].iloc[-20:],
                      'ema_short': ema_s.iloc[-20:],
                      'ema_long': ema_l.iloc[-20:],
                      'adx': adx.iloc[-20:]}))

# Suggest next steps printed for convenience
print('\nSuggestion: If ADX rarely exceeds your threshold, consider lowering adx_threshold (e.g., to 10-15), or relax EMA params (shorter/longer) to generate crossovers. I can apply a quick parameter sweep if you want.')